### 🧠 Pokročilé techniky a modely v detekci objektů

1. 📚 Přehled dalších modelů
🔷 Vizuální transformery (ViT – Vision Transformers)

- Založeny na architektuře transformerů známé z NLP.
- Místo konvolucí rozdělují obrázek na „patches“ a zpracovávají je jako sekvenci.
- *Výhoda*: lépe zachycují globální kontext.
- *Nevýhoda*: náročnější na data a výpočetní výkon.

*Příklady*:
- ViT (Google) – základní transformer pro obrázky.
- DETR (Facebook) – transformer pro detekci objektů.
- Swin Transformer – hierarchický transformer s posuvnými okny.

### 2. 🔁 Transfer learning a fine-tuning

#### 🧩 Co to je?
- Transfer learning: využití předtrénovaného modelu na nové úloze.
- Fine-tuning: doladění modelu na novém datasetu.

✅ *Výhody*:
- Rychlejší trénink.
- Vyšší přesnost na malých datasetech.
- Méně výpočetně náročné.

🧪 Ukázka v PyTorch:

In [ ]:
from torchvision.models import resnet18
import torch.nn as nn

# Načtení předtrénovaného modelu
model = resnet18(pretrained=True)

# Nahrazení výstupní vrstvy pro 3 třídy
model.fc = nn.Linear(model.fc.in_features, 3)

#### 3. 🔗 Kombinace modelů

##### 🧠 Příklad: Detekce + Klasifikace
- Detekční model (např. YOLO) najde objekty.
- Klasifikační model (např. ResNet) určí podtyp objektu.

🔄 Postup:
- Detekuj objekty (bounding boxy).
- Ořízni objekty z obrázku.
- Pošli je do klasifikátoru.

🧪 Ukázka:

In [ ]:
from PIL import Image
import torchvision.transforms as T

# Výstup z detekce: bbox = [x1, y1, x2, y2]
cropped = img.crop((x1, y1, x2, y2))

# Předzpracování pro klasifikátor
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])
input_tensor = transform(cropped).unsqueeze(0)

# Inference klasifikátoru
output = classifier(input_tensor)
predicted_class = output.argmax(dim=1)

#### výukový skript pro trénink a inference modelu DETR (Facebook)

In [ ]:
!pip install torch torchvision matplotlib
!git clone https://github.com/facebookresearch/detectron2.git

In [ ]:
# Ukázka anotace ve formátu COCO

{
  "images": [
    {"id": 1, "file_name": "image1.jpg", "width": 640, "height": 480}
  ],
  "annotations": [
    {"id": 1, "image_id": 1, "category_id": 1, "bbox": [100, 150, 200, 200], "area": 40000, "iscrowd": 0}
  ],
  "categories": [
    {"id": 1, "name": "object"}
  ]
}

In [ ]:
# Tréninkový skript (zjednodušený)

from detr.models.detr import build_detr
from detr.datasets.coco import make_coco_transforms
from torch.utils.data import DataLoader
import torch

# Dataset a transformace
transform = make_coco_transforms('train')
dataset = CustomCocoDataset('annotations.json', 'images/', transform=transform)
dataloader = DataLoader(dataset, batch_size=2, collate_fn=collate_fn)

# Model
model = build_detr(num_classes=2)  # 1 třída + background
model.to('cuda')

# Trénink
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
model.train()
for epoch in range(10):
    for images, targets in dataloader:
        images = [img.to('cuda') for img in images]
        targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        loss = sum(loss_dict.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
# skript pro Inferenci

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt

# Načtení modelu
model.eval()
img = Image.open("test.jpg").convert("RGB")
transform = T.Compose([T.Resize(800), T.ToTensor()])
img_tensor = transform(img).unsqueeze(0).to('cuda')

# Inference
outputs = model(img_tensor)
probs = outputs['pred_logits'].softmax(-1)[0, :, :-1]
boxes = outputs['pred_boxes'][0]

# Filtrace
keep = probs.max(-1).values > 0.7
boxes = boxes[keep]

# Zobrazení
def plot_results(pil_img, boxes):
    plt.imshow(pil_img)
    ax = plt.gca()
    for box in boxes:
        x, y, w, h = box
        ax.add_patch(plt.Rectangle((x, y), w, h, fill=False, color='red', linewidth=2))
    plt.axis('off')
    plt.show()

plot_results(img, boxes.cpu())